In [26]:
import numpy as np
import lightgbm as lgb

In [27]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [30]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference = lgb_train)

In [31]:
params = {
    'task' : 'train', # 수행할 작업
    'boosting_type' : 'gbdt', #rf(random forest) #gdbt(gradient boosted decision trees) 
    # dart(Dropouts meet Multiple Additive Regression Trees) # goss(Gradient-based One-Side Sampling)
    'objective' : 'binary', # loss관련 parameter
    # binary, multiclass, regression_l2, regression_l1, mape, poisson, quntile, huber, fair, gamma, lamdarank, tweedie
    'metric' : 'auc', # 평가지표
    'num_leaves' : 31, # max = 2^depth - 1
    'learning_rate' : 0.05, # 훈련량
    'feature_fraction' : 0.9, # 열 샘플링, 데이터 일부를 발췌해 다양성을 높힘
    'bagging_fraction' : 0.8, # 행 샘플링, 데이터 일부를 발췌해 다양성을 높힘
    'bagging_freq' : 5, # 데이터 업데이트 주기(default = 1), 몇번째 iteration에 해당하는 데이터를 업데이트 할 것인가
    'verbose' : 0 # < 0: Fatal, 0: Error (Warning), 1: Info, > 1: Debug
}

In [33]:
print('Start training....')
# train
gbm = lgb.train(params, 
                lgb_train, # 데이터 
                num_boost_round = 20, # 부스팅 횟수 
                valid_sets = lgb_eval, # 검증 데이터
                early_stopping_rounds = 5) # 5회 이상 성능 향상 없으면 멈춤

Start training....
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002578 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	valid_0's auc: 0.821017
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	valid_0's auc: 0.824482
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	valid_0's auc: 0.827976
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	valid_0's auc: 0.827568
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	valid_0's auc: 0.832227
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[6]	valid_0's auc: 0.83575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [35]:
print(gbm.best_iteration)
ypred = gbm.predict(X_test, num_iteration = gbm.best_iteration)

13


In [36]:
(sum((ypred>0.5) == y_test)) / 267.0

0.797752808988764